### Sentence Tokenize

This notebook includes sent tokenize techniques. 

In [1]:
#!pip install unicodedata2
#!pip install clean-text
#!pip install openpyxl

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from cleantext import clean
from nltk.stem import WordNetLemmatizer
from openpyxl.workbook import Workbook
nltk.download('tagsets')
nltk.download('punkt')
nltk.download('wordnet')
nltk.help.upenn_tagset('NNP')
nltk.help.upenn_tagset('NN')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
[nltk_data] Downloading package tagsets to /home/ubuntu/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...


NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...


[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/ubuntu/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
try:
  with open("tr6.txt", "r", encoding="utf-8") as file:
    text_data = file.read()
except:
  print("There is not such a file  or path is incorrect")

In [3]:
text_data_clean_brackets = re.sub('[\(\[\{].*?[\)\]\}]', '', text_data)

In [4]:
custom_char = ["-","#",":"]
for i in custom_char:
    text_data_clean_brackets = text_data_clean_brackets.replace(i, '')

In [5]:
text_data_clean_apos = re.sub(r"\'", "", string=text_data_clean_brackets)

In [6]:
text_data_clean = clean(text_data_clean_apos,
                        fix_unicode=True,
                        to_ascii=False,
                        lower=False,
                        normalize_whitespace=True,
                        no_line_breaks=True,
                        strip_lines=False,
                        keep_two_line_breaks=False,
                        no_urls=True,
                        no_emails=True,
                        no_phone_numbers=True,
                        no_numbers=True,
                        no_digits=True,
                        no_currency_symbols=True,
                        no_punct=False,
                        no_emoji=True,
                        replace_with_url='',
                        replace_with_email='',
                        replace_with_phone_number='',
                        replace_with_number='',
                        replace_with_digit='',
                        replace_with_currency_symbol='',
                        replace_with_punct=''
                        )

In [7]:
sentence = sent_tokenize(text_data_clean)

In [8]:
sentence_lower =  []
for i in sentence:
    sentence_lower.append(i.lower())

In [9]:
sent_token_df = pd.DataFrame(sentence_lower)

In [10]:
sent_token_df = sent_token_df.rename(columns={0:"sentence"})

In [11]:
sent_token_no_punc = sent_token_df.sentence.apply(lambda x: re.sub(pattern="[^\w\s]", repl="", string=x))

In [12]:
sent_token_no_punc_df = pd.DataFrame(data=sent_token_no_punc)

In [13]:
def space(sentence):
    a_1 = word_tokenize(sentence)
    a_2 = " ".join(a_1)
    return a_2

In [14]:
sent_token_no_punc_df.sentence = sent_token_no_punc_df.sentence.apply(lambda x : space(x))

In [15]:
sent_token_no_punc_df.drop(sent_token_no_punc_df[sent_token_no_punc_df.sentence == ""].index, inplace=True)

In [16]:
sent_token_no_punc_df = sent_token_no_punc_df.sentence.value_counts().sort_values(ascending=False)

In [17]:
sent_token_no_punc_df2 = pd.DataFrame(sent_token_no_punc_df)

In [18]:
sent_token_no_punc_df2 = sent_token_no_punc_df2.reset_index()

In [19]:
sent_token_no_punc_df2 = sent_token_no_punc_df2.rename(columns={"index":"sentence", "sentence":"frequency"})

In [20]:
total_value = sent_token_no_punc_df2.frequency.sum()

In [21]:
sent_token_no_punc_df2["ratio"] = (sent_token_no_punc_df2.frequency/total_value)*100

In [22]:
sent_token_no_punc_df2["cumul_ratio"] = np.cumsum(sent_token_no_punc_df2["ratio"])

In [23]:
#sent_token_no_punc_df3 = sent_token_no_punc_df2.head(500000)

In [24]:
def sentence_lenght(sentence):
    var1 = word_tokenize(sentence)
    if len(var1) <= 10:
        var2 = " ".join(var1)
        return var2
    else:
        return "sentence is bigger ten word"

In [25]:
sent_token_no_punc_df2.sentence = sent_token_no_punc_df2.sentence.apply(sentence_lenght)

In [26]:
sent_token_no_punc_df2 = sent_token_no_punc_df2.reset_index(drop=True)

In [27]:
sent_token_no_punc_df2.to_csv("Sentence_Tokenize_TR6.csv", index=False)